In [1]:
PHI = {}

p = 5

In [2]:
from fractions import Fraction

In [32]:
from copy import deepcopy


def omega_0(x):
    if (x >= 0 and x < p - 1):
        return x
    else:
        return 0
    
omega = omega_0
    
    
def z_p_func_template():
    return {'blocks': []}

def phi_by_omega():
    phi = z_p_func_template()

    for i in range(p-1):
        phi['blocks'] += [(['z', i+1], omega(i))]

    return phi



def plus_one(y0):
    Y = [y0]

    k = 0
    t = 1
    explode = False
    for digit in y0:
        if digit == 'z' and t > 0:
            explode = True
            break       
        t = 0 if (digit + t < p) else 1
        if t == 0:
            break
        k += 1
        
    if explode:
        y1 = deepcopy(y0)
        y1.remove('z')
        y2 = deepcopy(y0)
        y2.insert(k, 0)
        Y = [y1, y2]
    
    Z = []
    for y in Y:
        z = []
        t = 1
        for digit in y:
            if t > 0:
                digit2 = (digit + t) % p
                t = 0 if (digit + t < p) else 1
                z.append(digit2) 
            else:
                z.append(digit)
        Z.append(z)
    
    return Z 
        

def plus_one_to_block(block):
    a = plus_one(block[0])
    b = []
    for s in a:
        b.append((s, block[1]))
    return b


def shift(f):
    f2 = deepcopy(f)
    f2['blocks'] = []
    for block in f['blocks']:
        b = plus_one_to_block(block)
        f2['blocks'] += b        
    return f2
        

def generate_phi_m(m_max):
    for m in range(1, m_max + 1):
        if m in PHI:
            continue
        if (m == 1):
            PHI[1] = phi_by_omega()
            return
        PHI[m] = add(PHI[1], shift(PHI[1]))
        
        
def prob(s):
    pr = 1
    for digit in s:
        pr *= (Fraction(p, p - 1) if digit == 'z' else Fraction(1, p))
    return pr

    
def integrate(f):
    integral = 0
    for block in f['blocks']:
        integral += prob(block[0]) * block[1]
    return integral


def add(f1, f2):
    f = deepcopy(f1)
    f['blocks'] = []
    for block1 in f1['blocks']:
        for block2 in f2['blocks']:
            a = intersect(block1[0], block2[0])
            for s in a:
                f['blocks'].append((s, block1[1] + block2[1]))
    return f


def simple_intersect(s1, s2):
    assert ('z' not in s1 and 'z' not in s2), 'one of s1, s2 contains z'
    # assert (len(s1) > 0 and len(s2) > 0), 'one of s1, s2 is empty'

    n1 = len(s1)
    n2 = len(s2)

    if(n1 == 0):
        return [s2]
    
    if (n2 == 0):
        return [s1]
    
    if(n2 > n1):
        s1, s2 = s2, s1
        n1, n2 = n2, n1
        
    # now s1 is the longest
    
    s = []
    for i in range(n2):
        if(s1[i] != s2[i]):
            return []
    return [s1]


def intersect1(seq_with_z, usual_seq, z_index):
    seq_before_z = seq_with_z[: z_index]
    seq_after_z = seq_with_z[z_index + 1: ]
    
    s = simple_intersect(seq_before_z, usual_seq)
    
    if(len(s) == 0):
        return []
    
    if(len(usual_seq) <= z_index):
        return [seq_with_z]
    
    t = seq_after_z[0]
    k = 0
    res = []

    
    for j in range(z_index, len(usual_seq)):
        if(usual_seq[j] != 0):
            if(usual_seq[z_index] == t):
                return [usual_seq]
            else:
                return []
        else:
            pass
        
    return [usual_seq + seq_after_z]
    
    
def intersect2(s1, s2, k1, k2):
    if(k1 > k2):
        s1, s2, k1, k2 = s2, s1, k2, s1
    
    n1 = len(s1)
    n2 = len(s2)
    
    # s1 has 'z' in earlier position
    
    a = [] 
    s1_before_z = s1[: k1]
    zeros = []
    s1_after_z = s1[k1 + 1: ]
    
    for n_zeros in range(k2 - k1):
        a += intersect1(usual_seq=(s1_before_z + zeros + s1_after_z), seq_with_z=s2, z_index=k2)
        zeros += [0]
        
    if(s1[k1 + 1] == s2[k2 + 1]):
        if(s1[: k1] + zeros == s2[: k2]):
            a += [s1]

    return a
    
def intersect(s1, s2):
    k1 = -1
    k2 = -1    
    
    for i in range(len(s1)):
        if (s1[i] == 'z'):
            k1 = i
            
    for i in range(len(s2)):
        if (s2[i] == 'z'):
            k2 = i      
    
    if (k1 == -1 and k2 == -1):
        return simple_intersect(s1, s2)
    
    if (k1 == -1 and k2 != -1):
        return intersect1(seq_with_z=s2, usual_seq=s1, z_index=k2)
    
    if (k1 != -1 and k2 == -1):
        return intersect1(seq_with_z=s1, usual_seq=s2, z_index=k1)
    
    if (k1 != -1 and k2 != -1):
        return intersect2(s1, s2, k1, k2)

In [26]:
def k_fold_shift(f, k):
    if k <= 0:
        return f
    return shift(f) if k == 1 else shift(k_fold_shift(f, k-1))

In [3]:
generate_phi_m(1)

In [4]:
print(PHI)

{1: {'blocks': [(['z', 1], 0), (['z', 2], 1), (['z', 3], 2), (['z', 4], 3)]}}


In [19]:
PHI[1]['blocks']

[(['z', 1], 0), (['z', 2], 1), (['z', 3], 2), (['z', 4], 3)]

In [20]:
shift(PHI[1])['blocks']

[([2], 0),
 ([1, 'z', 1], 0),
 ([3], 1),
 ([1, 'z', 2], 1),
 ([4], 2),
 ([1, 'z', 3], 2),
 ([0], 3),
 ([1, 'z', 4], 3)]

In [21]:
shift(shift(PHI[1]))['blocks']

[([3], 0),
 ([2, 'z', 1], 0),
 ([4], 1),
 ([2, 'z', 2], 1),
 ([0], 2),
 ([2, 'z', 3], 2),
 ([1], 3),
 ([2, 'z', 4], 3)]

In [22]:
shift(shift(shift(PHI[1])))['blocks']

[([4], 0),
 ([3, 'z', 1], 0),
 ([0], 1),
 ([3, 'z', 2], 1),
 ([1], 2),
 ([3, 'z', 3], 2),
 ([2], 3),
 ([3, 'z', 4], 3)]

In [24]:
k_fold_shift(PHI[1], 5)

{'blocks': [([1], 0),
  ([0, 2], 0),
  ([0, 1, 'z', 1], 0),
  ([2], 1),
  ([0, 3], 1),
  ([0, 1, 'z', 2], 1),
  ([3], 2),
  ([0, 4], 2),
  ([0, 1, 'z', 3], 2),
  ([4], 3),
  ([0, 0], 3),
  ([0, 1, 'z', 4], 3)]}

In [28]:
integrate(PHI[1])

TypeError: can't multiply sequence by non-int of type 'float'

In [33]:
[integrate(k_fold_shift(PHI[1], x)) for x in range(1,10)]

[1.5,
 1.5,
 1.5,
 1.5,
 1.5000000000000002,
 1.5000000000000002,
 1.5000000000000002,
 1.5000000000000002,
 1.5000000000000002]

100/99


In [2]:
a = [1, 2, 3]
a.append([4, 5])
a

[1, 2, 3, [4, 5]]

In [3]:
[1,2]==[1,2]

True